In [ ]:
import assemblyai as aai

aai.settings.api_key = "9dc0beed5a1b4f8ebb6b32694b400171"

In [3]:
import pyaudio
import numpy as np
import wave
import threading

# Global variable to control recording
recording = False

def record_audio(filename):
    global recording
    print("Recording started. Press 'Enter' to stop recording...")
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 2
    RATE = 44100
    audio_data = []  # Initialize audio data array

    # Start recording
    recording = True
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    while recording:
        # Record audio in chunks
        data = stream.read(CHUNK)
        audio_data.append(data)

    # Stop recording
    stream.stop_stream()
    stream.close()
    p.terminate()
    print("Recording stopped.")

    # Save the recorded audio to a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(audio_data))
    wf.close()

def start_recording(filename):
    global recording
    if not recording:
        recording = True
        threading.Thread(target=record_audio, args=(filename,), daemon=True).start()

def stop_recording():
    global recording
    recording = False

In [ ]:
audio_url =input("enter file name")
while True:
    user_input = input("Press 'r' to start recording, 's' to stop recording, or 'q' to quit: ").lower()
    if user_input == 'r':
        start_recording(audio_url)
    elif user_input == 's':
        stop_recording()
    elif user_input == 'q':
        break
else:
    print("Invalid choice. Exiting.")

enter file namelak.wav


In [ ]:
import wave
from mutagen.mp3 import MP3
from mutagen.mp4 import MP4
from mutagen.flac import FLAC
from mutagen.wavpack import WavPack
from mutagen.oggvorbis import OggVorbis

def get_audio_duration(filename):
    if filename.endswith('.mp3'):
        audio = MP3(filename)
    elif filename.endswith('.mp4'):
        audio=MP4(filename)
    elif filename.endswith('.flac'):
        audio = FLAC(filename)
    elif filename.endswith('.wv'):
        audio = WavPack(filename)
    elif filename.endswith('.ogg'):
        audio = OggVorbis(filename)
    elif filename.endswith('.wav'):
        with wave.open(filename, 'rb') as audio_file:
            frames = audio_file.getnframes()
            rate = audio_file.getframerate()
            duration_seconds = frames / float(rate)
        return duration_seconds
    else:
        raise ValueError("Unsupported audio format")

    duration_seconds = audio.info.length
    return duration_seconds

def format_duration(duration):
    hours = int(duration / 3600)
    minutes = int((duration % 3600) / 60)
    seconds = int(duration % 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

audio_file = audio_url 
duration_seconds = get_audio_duration(audio_file)
duration_formatted = format_duration(duration_seconds)
print("Audio duration:", duration_formatted)


In [ ]:
print("do you want speaker labels?")
print("NOTE: number of speakers should be more than 1 for speaker labels to be implemented")
choice=input()
speaker_label= bool(choice=='yes')
print("do you want text content to be in paragraphs or sentences?")
choice1=input()
para_label=bool(choice1=='paragraphs')

In [ ]:
print("do you want to transcribe a portion of your file?")
choice2=input()
check=bool(choice2=='yes')
if check:
    start=input("enter start time(hh:mm:ss)")
    start_hours,start_minutes,start_seconds = map(int, start.split(':'))
    end=input("enter end time(hh:mm:ss)")
    end_hours,end_minutes,end_seconds = map(int, end.split(':'))
else:
    start_hours,start_minutes,start_seconds=0,0,0
    end_hours,end_minutes,end_seconds=map(int, duration_formatted.split(':'))
start_milliseconds = (start_hours * 60 * 60 * 1000) + (start_minutes * 60 * 1000) + (start_seconds * 1000)
end_milliseconds = (end_hours * 60 * 60 * 1000) + (end_minutes * 60 * 1000) + (end_seconds * 1000)


In [ ]:
def speech_to_text(url,speaker_label, para_label, start_milliseconds, end_milliseconds):
    config = aai.TranscriptionConfig(auto_highlights=True,speaker_labels=speaker_label, audio_start_from=start_milliseconds, audio_end_at=end_milliseconds)
    transcriber = aai.Transcriber(config=config)
    transcript = transcriber.transcribe(url)

    if transcript.status == aai.TranscriptStatus.error:
        print(transcript.error)
        return ""
    else:
        formatted_transcript = ""  
        highlights=""
        
        for result in transcript.auto_highlights.results:
            highlights+= f"Highlight: {result.text}, Count: {result.count}, Rank: {result.rank}, Timestamps: {result.timestamps}\n"
            
        if speaker_label:
            speakers = set(utterance.speaker for utterance in transcript.utterances)
            if len(speakers) == 1:
                if para_label:
                    paragraphs = transcript.get_paragraphs()
                    for paragraph in paragraphs:
                        formatted_transcript += paragraph.text + "\n"
                else:
                    sentences = transcript.get_sentences()
                    for sentence in sentences:
                        formatted_transcript += sentence.text + "\n"
            else:
                for utterance in transcript.utterances:
                    formatted_transcript += f"Speaker {utterance.speaker}: "
                    sentences = nltk.sent_tokenize(utterance.text)
                    if para_label:
                        paragraphs = [' '.join(sentences[i:i+2]) for i in range(0, len(sentences), 2)]
                        for paragraph in paragraphs:
                            formatted_transcript += paragraph + "\n"
                    else:
                        for sentence in sentences:
                            formatted_transcript += sentence + "\n"
        else:
            if para_label:
                paragraphs = transcript.get_paragraphs()
                for paragraph in paragraphs:
                    formatted_transcript += paragraph.text + "\n"  
            else:
                sentences = transcript.get_sentences()
                for sentence in sentences:
                    formatted_transcript += sentence.text + "\n"  

        return formatted_transcript,highlights


In [ ]:
text,highlights = speech_to_text(audio_url,speaker_label, para_label, start_milliseconds, end_milliseconds)
print(text)